In [1]:
#pip install yfinance --upgrade --no-cache-dir
#pip install psycopg2
#pip install SQLAlchemy

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/14/03/3c854ca3e02eedf614abba4b2e177c469bf3af58207fa30d5098c5d652fe/yfinance-0.2.37-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for pytz>=2022.5 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 964.8 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 342.4 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requi

In [98]:
import yfinance as yf
import pandas as pd
import psycopg2
from getpass import getpass
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [2]:
tickers = yf.Tickers('msft aapl goog tsla amzn meta')

In [120]:
companies = ('MSFT','AAPL','GOOG','TSLA','AMZN','META')
historical = pd.DataFrame()
for i in  range(len(companies)):
    df = tickers.tickers[companies[i]].history(period="5y")
    df['symbol'] = tickers.tickers[companies[i]].info['symbol']
    df['short_name'] = tickers.tickers[companies[i]].info['shortName']
    df['long_name'] = tickers.tickers[companies[i]].info['longName']
    df['address'] = tickers.tickers[companies[i]].info['address1']
    df['city'] = tickers.tickers[companies[i]].info['city']
    df['state'] = tickers.tickers[companies[i]].info['state']
    df['zip'] = tickers.tickers[companies[i]].info['zip']
    df['country'] = tickers.tickers[companies[i]].info['country']
    df['phone'] = tickers.tickers[companies[i]].info['phone']
    df['website'] = tickers.tickers[companies[i]].info['website']
    df['industry'] = tickers.tickers[companies[i]].info['industry']
    df['sector'] = tickers.tickers[companies[i]].info['sector']
    historical = pd.concat([historical,df], ignore_index=False)
historical = historical.reset_index()
subrogate_key = historical['Date'].apply(lambda x: dt.datetime.strftime(x,"%Y%m%d%H%M%S")) + historical['symbol']
historical['stock_id'] = subrogate_key
historical['Date'] = historical['Date'].apply(lambda x: dt.datetime.strptime(dt.datetime.strftime(x,"%Y-%m-%d"),"%Y-%m-%d"))
historical = historical.rename(columns={'Open':'open_value', 'Close':'close_value','High':'high_value', 'Low':'low_value', 'Stock Splits':'stock_splits'})



In [69]:
historical.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,symbol,short_name,...,city,state,zip,country,phone,website,industry,sector,business_description,stock_id
0,2019-03-29 00:00:00-04:00,112.389941,112.627914,111.333339,112.266197,25399800,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,Microsoft Corporation develops and supports so...,20190329000000MSFT
1,2019-04-01 00:00:00-04:00,113.227595,113.379902,112.418488,113.294228,22789100,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,Microsoft Corporation develops and supports so...,20190401000000MSFT
2,2019-04-02 00:00:00-04:00,113.332289,113.732089,112.818266,113.456039,18142300,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,Microsoft Corporation develops and supports so...,20190402000000MSFT
3,2019-04-03 00:00:00-04:00,114.093831,114.636409,113.417988,114.198540,22860700,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,Microsoft Corporation develops and supports so...,20190403000000MSFT
4,2019-04-04 00:00:00-04:00,114.322272,114.446023,112.685016,113.617874,20112800,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,Microsoft Corporation develops and supports so...,20190404000000MSFT


In [151]:
host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="gustavo_hiram_gutierrez_coderhouse"
pwd = '' #getpass()

try:
    conn = psycopg2.connect(
        host=host,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conexión a Redshift Existosa!")
    
except Exception as e:
    print("No es posible conectarse a Redshift")
    print(e)

Conexión a Redshift Existosa!


In [155]:
create_table = ''' CREATE TABLE IF NOT EXISTS gustavo_hiram_gutierrez_coderhouse.stock_market  (
    stock_id varchar(25) NOT NULL PRIMARY KEY,
    symbol VARCHAR(100) NOT NULL,
    short_name VARCHAR(100) NOT NULL,
    long_name VARCHAR(100) NOT NULL,
    address VARCHAR(100) NOT NULL,
    city VARCHAR(100) NOT NULL,
    state VARCHAR(100) NOT NULL,
    zip VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    phone VARCHAR(100) NOT NULL,
    website VARCHAR(100) NOT NULL,
    industry VARCHAR(100) NOT NULL,
    sector VARCHAR(100) NOT NULL,
    date date NOT NULL,
    open_value DECIMAL NOT NULL,
    high_value DECIMAL NOT NULL,
    low_value DECIMAL NOT NULL,
    close_value DECIMAL NOT NULL,
    volume integer NOT NULL,
    dividends DECIMAL NOT NULL,
    stock_splits DECIMAL NOT NULL
)'''

with conn.cursor() as cur:
    cur.execute(create_table)
    conn.commit()
print('Tabla Creada de Manera Exitosa!')

Tabla Creada de Manera Exitosa!


In [148]:
#Existen dos Métodos de ingestar la data a Redshift

# Método 1
conn_string = URL.create(
    'postgresql',
    username=user,
    password=pwd,
    host=host,
    port='5439',
    database=data_base)
engine = create_engine(conn_string)
with engine.connect() as conn:
    companies = ('MSFT','AAPL','GOOG','TSLA','AMZN','META')
    for i in  range(len(companies)):
        engine = create_engine(conn_string)
        historical[historical['symbol']== companies[i]].to_sql('stock_market1', con=conn, index=False, if_exists='append')
        print('Data correctamente insertada para ', companies[i])

Data correctamente insertada para  MSFT
Data correctamente insertada para  AAPL
Data correctamente insertada para  GOOG
Data correctamente insertada para  TSLA
Data correctamente insertada para  AMZN
Data correctamente insertada para  META


In [156]:
# Método 2
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO gustavo_hiram_gutierrez_coderhouse.stock_market (Date, open_value, high_value, low_value,
          close_value, Volume, Dividends, stock_splits,symbol,short_name,long_name,address,city,state,zip,country,phone,
          website,industry,sector,stock_id)
        VALUES %s
        ''',
        [tuple(row) for row in historical.values],
        page_size=len(historical)
    )
    conn.commit()
print('Data correctamente insertada!')

Data correctamente insertada!
